In [ ]:
# Importações
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    roc_auc_score
)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast

# Cross validation
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, train_test_split

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Define funções essenciais

# Variáveis para padronização das métricas
metrics = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "roc_auc": roc_auc_score
}

def evaluate_metrics(y_true, y_pred, y_pred_proba, metrics=metrics):
    # Calculate scores
    scores = {}
    for name, metric in metrics.items():
        # Handle metrics that require probabilities (like ROC AUC)
        if name == 'roc_auc':
            scores[name] = metric(y_true, y_pred_proba)
        else:
            scores[name] = metric(y_true, y_pred)
    return scores

In [ ]:
# Importa Dataframe contendo títulos de notícias relacionadas a saúde
df = pd.read_csv('dados/df_final.zip')

# Re-transforma as listas em listas
df['title'] = df['title'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['corpo_texto'] = df['corpo_texto'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

display(df)

In [ ]:
# Separa porção para validação
df_main, df_validation = train_test_split(df, test_size=0.25, random_state=42, stratify=df['classe'])

display(df_main, df_validation)

In [ ]:
# Checa distribuição de classes entre dados separados
print(f'Distribuição para o df principal: {df_main['classe'].value_counts()[0]/\
      df_main['classe'].value_counts()[1]}')
print(f'\nDistribuição para o df de validação: {df_validation['classe'].value_counts()[0]/\
      df_validation['classe'].value_counts()[1]}')

In [ ]:
# Inicializa o vetorizador de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Criação de vetores
X = df_main['title'].apply(lambda x: ' '.join(x))
y = df_main['classe']

print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)

### Árvore de Decisão

In [ ]:
# Cria modelo e define pipeline com TF-IDF
modelo_ad = DecisionTreeClassifier()
pipeline_ad = Pipeline([('TF-IDF', tfidf_vectorizer), ('modelo', modelo_ad)])

# Validação cruzada
resultados_ad = cross_validate(pipeline_ad, X, y, scoring=[score for score in metrics.keys()])

acc_ad = resultados_ad['test_accuracy'].mean()
acc_ad_std = np.std(resultados_ad['test_accuracy'])
prec_ad = resultados_ad['test_precision'].mean()
prec_ad_std = np.std(resultados_ad['test_precision'])
rocauc_ad = resultados_ad['test_roc_auc'].mean()
rocauc_ad_std = np.std(resultados_ad['test_roc_auc'])
print(f'Acurácia: {acc_ad:.3f}±{acc_ad_std:.3f}\
    \nPrecisão: {prec_ad:.3f}±{prec_ad_std:.3f}\
    \nROC-AUC: {rocauc_ad:.3f}±{rocauc_ad_std:.3f}')

### Floresta Aleatória

In [ ]:
# Cria modelo
modelo_rf = RandomForestClassifier()
pipeline_rf = Pipeline([('TF-IDF', tfidf_vectorizer), ('modelo', modelo_rf)])

# Validação cruzada
resultados_rf = cross_validate(pipeline_rf, X, y, scoring=[score for score in metrics.keys()])

acc_rf = resultados_rf['test_accuracy'].mean()
acc_rf_std = np.std(resultados_rf['test_accuracy'])
prec_rf = resultados_rf['test_precision'].mean()
prec_rf_std = np.std(resultados_rf['test_precision'])
rocauc_rf = resultados_rf['test_roc_auc'].mean()
rocauc_rf_std = np.std(resultados_rf['test_roc_auc'])
print(f'Acurácia: {acc_rf:.3f}±{acc_rf_std:.3f}\
    \nPrecisão: {prec_rf:.3f}±{prec_rf_std:.3f}\
    \nROC-AUC: {rocauc_rf:.3f}±{rocauc_rf_std:.3f}')

### Regressão Logística

In [ ]:
# Cria modelo
modelo_lr = LogisticRegression()
pipeline_lr = Pipeline([('TF-IDF', tfidf_vectorizer), ('modelo', modelo_lr)])

# Validação cruzada
resultados_lr = cross_validate(pipeline_lr, X, y, scoring=[score for score in metrics.keys()])

acc_lr = resultados_lr['test_accuracy'].mean()
acc_lr_std = np.std(resultados_lr['test_accuracy'])
prec_lr = resultados_lr['test_precision'].mean()
prec_lr_std = np.std(resultados_lr['test_precision'])
rocauc_lr = resultados_lr['test_roc_auc'].mean()
rocauc_lr_std = np.std(resultados_lr['test_roc_auc'])
print(f'Acurácia: {acc_lr:.3f}±{acc_lr_std:.3f}\
    \nPrecisão: {prec_lr:.3f}±{prec_lr_std:.3f}\
    \nROC-AUC: {rocauc_lr:.3f}±{rocauc_lr_std:.3f}')

### SVM

In [ ]:
# Parâmetros
C = 77
kernel = 'rbf'
gamma = 0.01

# Cria modelo
modelo_svm = svm.SVC(C=C, kernel=kernel, gamma=gamma, probability=True)
pipeline_svm = Pipeline([('TF-IDF', tfidf_vectorizer), ('modelo', modelo_svm)])

# Validação cruzada
resultados_svm = cross_validate(pipeline_svm, X, y, scoring=[score for score in metrics.keys()])

acc_svm = resultados_svm['test_accuracy'].mean()
acc_svm_std = np.std(resultados_svm['test_accuracy'])
prec_svm = resultados_svm['test_precision'].mean()
prec_svm_std = np.std(resultados_svm['test_precision'])
rocauc_svm = resultados_svm['test_roc_auc'].mean()
rocauc_svm_std = np.std(resultados_svm['test_roc_auc'])
print(f'Acurácia: {acc_svm:.3f}±{acc_svm_std:.3f}\
    \nPrecisão: {prec_svm:.3f}±{prec_svm_std:.3f}\
    \nROC-AUC: {rocauc_svm:.3f}±{rocauc_svm_std:.3f}')

In [ ]:
# Definindo os dados
accuracy_list = [acc_ad, acc_rf, acc_lr, acc_svm]
precision_list = [prec_ad, prec_rf, prec_lr, prec_svm]
rocauc_list = [rocauc_ad, rocauc_rf, rocauc_lr, rocauc_svm]
acc_std_list = [acc_ad_std, acc_rf_std, acc_lr_std, acc_svm_std]
prec_std_list = [prec_ad_std, prec_rf_std, prec_lr_std, prec_svm_std]
rocauc_std_list = [rocauc_ad_std, rocauc_rf_std, rocauc_lr_std, rocauc_svm_std]
model_labels = ['Árvore de Decisão', 'Floresta Aleatória', 'Regressão Logística', 'SVM']
colors = ['red', 'orange', 'pink', 'purple']

# Posições das barras para os grupos
x = np.arange(3)  # Precisão, Acurácia e ROC-AUC
width = 0.15      # Largura das barras

# Criando o gráfico sem zoom
fig, ax = plt.subplots(figsize=(10, 6))
for i in range(len(model_labels)):
    # Precisão
    ax.bar(x[0] + i * width - (1.5 * width), precision_list[i], width, label=model_labels[i], color=colors[i])
    ax.errorbar(x[0] + i * width - (1.5 * width), precision_list[i], yerr=prec_std_list[i], fmt='o', color='black', capsize=5)
    # Acurácia
    ax.bar(x[1] + i * width - (1.5 * width), accuracy_list[i], width, color=colors[i])
    ax.errorbar(x[1] + i * width - (1.5 * width), accuracy_list[i], yerr=acc_std_list[i], fmt='o', color='black', capsize=5)
    # ROC-AUC
    ax.bar(x[2] + i * width - (1.5 * width), rocauc_list[i], width, color=colors[i])
    ax.errorbar(x[2] + i * width - (1.5 * width), rocauc_list[i], yerr=rocauc_std_list[i], fmt='o', color='black', capsize=5)

# Configurando os rótulos e título
ax.set_xlabel('Métrica')
ax.set_ylabel('Valores')
ax.set_title('Precisão, Acurácia e ROC-AUC dos Modelos para os Títulos (Sem Zoom)')
ax.set_xticks(x)
ax.set_xticklabels(['Precisão', 'Acurácia', 'ROC-AUC'])
ax.legend()

# Exibindo o gráfico sem zoom
plt.show()

# Criando o gráfico com zoom
fig, ax = plt.subplots(figsize=(10, 6))
for i in range(len(model_labels)):
    # Precisão
    ax.bar(x[0] + i * width - (1.5 * width), precision_list[i], width, label=model_labels[i], color=colors[i])
    ax.errorbar(x[0] + i * width - (1.5 * width), precision_list[i], yerr=prec_std_list[i], fmt='o', color='black', capsize=5)
    # Acurácia
    ax.bar(x[1] + i * width - (1.5 * width), accuracy_list[i], width, color=colors[i])
    ax.errorbar(x[1] + i * width - (1.5 * width), accuracy_list[i], yerr=acc_std_list[i], fmt='o', color='black', capsize=5)
    # ROC-AUC
    ax.bar(x[2] + i * width - (1.5 * width), rocauc_list[i], width, color=colors[i])
    ax.errorbar(x[2] + i * width - (1.5 * width), rocauc_list[i], yerr=rocauc_std_list[i], fmt='o', color='black', capsize=5)

# Configurando os rótulos e título
ax.set_xlabel('Métrica')
ax.set_ylabel('Valores')
ax.set_title('Precisão, Acurácia e ROC-AUC dos Modelos para os Títulos (Com Zoom)')
ax.set_xticks(x)
ax.set_xticklabels(['Precisão', 'Acurácia', 'ROC-AUC'])
ax.legend()

# Definindo o limite do eixo y para dar o "zoom"
plt.ylim(0.75, 1)

# Exibindo o gráfico com zoom
plt.show()

### Teste com dados separados pra validação

In [ ]:
# Inicializa o vetorizador de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Criação de vetores
X_val = df_validation['title'].apply(lambda x: ' '.join(x))
y_val = df_validation['classe'].to_numpy()

print("Dimensões de X:", X_val.shape)
print("Dimensões de y:", y_val.shape)

In [ ]:
# Calcula score de cada método para df de validação

pipelines = [pipeline_ad, pipeline_rf, pipeline_lr, pipeline_svm]

accuracy_list = []
precision_list = []
rocauc_list = []


for pipeline in pipelines:
    pipeline.fit(X,y)
    y_val_pred = pipeline.predict(X_val)
    y_val_pred_proba = pipeline.predict_proba(X_val)[:,1]
    scores = evaluate_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    accuracy_list.append(scores['accuracy'])
    precision_list.append(scores['precision'])
    rocauc_list.append(scores['roc_auc'])
    
    print(f'{str(pipeline[1])}:\n    Acurácia: {scores['accuracy']:.3f}\
        \n    Precisão: {scores['precision']:.3f}\n    ROC-AUC: {scores['roc_auc']:.3f}\n')

In [ ]:
# Positions for the groups of bars
x = np.arange(3)  # One point for "Precision", "Accuracy", and "ROC-AUC"
width = 0.15  # Width of the bars

# Creating the plot without zoom
fig, ax = plt.subplots(figsize=(10, 6))
for i in range(len(model_labels)):
    # Precision
    ax.bar(x[0] + i * width - (1.5 * width), precision_list[i], width, label=model_labels[i], color=colors[i])
    # Accuracy
    ax.bar(x[1] + i * width - (1.5 * width), accuracy_list[i], width, color=colors[i])
    # ROC-AUC
    ax.bar(x[2] + i * width - (1.5 * width), rocauc_list[i], width, color=colors[i])

# Configuring labels and title
ax.set_xlabel('Métrica')
ax.set_ylabel('Valores')
ax.set_title('Validação - Precisão, Acurácia e ROC-AUC dos Modelos para os Títulos (Sem Zoom)')
ax.set_xticks(x)
ax.set_xticklabels(['Precisão', 'Acurácia', 'ROC-AUC'])
ax.legend()

# Displaying the plot without zoom
plt.show()

# Creating the plot with zoom
fig, ax = plt.subplots(figsize=(10, 6))
for i in range(len(model_labels)):
    # Precision
    ax.bar(x[0] + i * width - (1.5 * width), precision_list[i], width, label=model_labels[i], color=colors[i])
    # Accuracy
    ax.bar(x[1] + i * width - (1.5 * width), accuracy_list[i], width, color=colors[i])
    # ROC-AUC
    ax.bar(x[2] + i * width - (1.5 * width), rocauc_list[i], width, color=colors[i])

# Configuring labels and title
ax.set_xlabel('Métrica')
ax.set_ylabel('Valores')
ax.set_title('Validação - Precisão, Acurácia e ROC-AUC dos Modelos para os Títulos (Com Zoom)')
ax.set_xticks(x)
ax.set_xticklabels(['Precisão', 'Acurácia', 'ROC-AUC'])
ax.legend()

# Setting y-axis limit for zoom
plt.ylim(0.7, 1)

# Displaying the plot with zoom
plt.show()